In [14]:
%run ../common_functions.py

read_dataset()
read_all_datasets()
read_processed_data()
combine_text_columns()
split_train_test()
extract_data()
evaluate_model()
detect_missing_values()


In [21]:
from sklearn.linear_model import LogisticRegression
from sentence_transformers import SentenceTransformer

## Load in Dataset

In [10]:
data = read_processed_data(augmented=False)

## Preprocess

## Split data

In [12]:
train_data, test_data = split_train_test(data=data, test_size=0.2)

## Logistic Regression

### Extract data and label mapping

In [20]:
X_train, y_train, X_test, y_test = extract_data(train_df=train_data, test_df=test_data, text_col="text", label_col="label")

### Transform to Vectors

In [22]:
embedder = SentenceTransformer("all-mpnet-base-v2")

X_train_emb = embedder.encode(
    X_train, batch_size=32, show_progress_bar=True
)
X_test_emb  = embedder.encode(X_test, batch_size=32)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.4k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Cancellation requested; stopping current tasks.


KeyboardInterrupt: 

### Train Logistic Regression Classifier

In [10]:
model = LogisticRegression(
    C=0.01,
    class_weight='balanced',
    max_iter=1000,
    random_state=42,
    penalty='l2',
    solver="liblinear"
)
model.fit(X_train_emb, y_train)

LogisticRegression(C=0.01, class_weight='balanced', max_iter=1000,
                   random_state=42, solver='liblinear')

### Predict and Evaluate

In [12]:
y_pred = model.predict(X_test_emb)

evaluate_model(y_test=y_test, y_pred=y_pred)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.87      0.93      1919
           1       0.24      0.99      0.39        81

    accuracy                           0.88      2000
   macro avg       0.62      0.93      0.66      2000
weighted avg       0.97      0.88      0.91      2000


Overall Metrics:
Accuracy       : 0.8760
Precision (avg): 0.9688
Recall    (avg): 0.8760
F1 Score  (avg): 0.9091

Confusion Matrix:
[[1672  247]
 [   1   80]]
